In [1]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

In [2]:
# 데이터 로드
data = pd.read_csv('merged_data.csv', encoding='utf-8')

In [3]:
data

,stn_id,borrowed_hour,borrowed_day,is_holiday,borrowed_num_nearby,강수량(mm),wind_chill,stn_gu,nearby_id,borrowed_date,borrowed_num
0,ST-10,0,6,1,2,0.0,21.787650,마포구,ST-2167,20221001,2
1,ST-10,1,6,1,1,0.0,20.724511,마포구,ST-2167,20221001,2
2,ST-10,2,6,1,2,0.0,20.222465,마포구,ST-2167,20221001,1
3,ST-10,3,6,1,0,0.0,19.789024,마포구,ST-2167,20221001,0
4,ST-10,4,6,1,2,0.0,20.661831,마포구,ST-2167,20221001,0
...,...,...,...,...,...,...,...,...,...,...,...
9647433,ST-1445,20,2,0,1,0.0,8.507538,용산구,ST-1328,20230228,0
9647434,ST-1445,21,2,0,0,0.0,9.551939,용산구,ST-1328,20230228,3
9647435,ST-1445,22,2,0,0,0.0,8.219911,용산구,ST-1328,20230228,0
9647436,ST-1445,23,2,0,0,0.0,9.181400,용산구,ST-1328,20230228,1


In [4]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]

In [5]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]

C:\Users\user\AppData\Local\Temp\ipykernel_18108\581326598.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature] = pd.factorize(data[feature])[0]
C:\Users\user\AppData\Local\Temp\ipykernel_18108\581326598.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature] = pd.factorize(data[feature])[0]


In [6]:
# 훈련-테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)

# LGBM Model

In [7]:
# LightGBM 모델
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 80,
    'learning_rate': 0.1,
    'feature_fraction': 1.0
}

In [8]:
train_data_lgb = lgb.Dataset(X_train, label=y_train)
test_data_lgb = lgb.Dataset(X_test, label=y_test, reference=train_data_lgb)

lgb_model = lgb.train(lgb_params, train_data_lgb, num_boost_round=1000, valid_sets=[test_data_lgb, train_data_lgb], callbacks=[
    lgb.early_stopping(stopping_rounds=3, verbose=100),
])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.847047 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1112
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 9
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.92098	valid_0's rmse: 1.97004


# XGBoost Model

In [9]:
# XGBoost 모델
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'booster': 'gbtree',
    'learning_rate': 0.1,
    'max_depth': 13,
    'subsample':0.8
}

In [10]:
train_data_xgb = xgb.DMatrix(X_train, label=y_train)
test_data_xgb = xgb.DMatrix(X_test, label=y_test)

xgb_model = xgb.train(xgb_params, train_data_xgb, num_boost_round=1000, evals=[(test_data_xgb, 'eval')], early_stopping_rounds=3, verbose_eval=100)


[0]	eval-rmse:2.79100
[100]	eval-rmse:1.97047
[200]	eval-rmse:1.91868
[250]	eval-rmse:1.91346


In [11]:
# 예측 생성
y_pred_train_lgb = lgb_model.predict(X_train, num_iteration=lgb_model.best_iteration)
y_pred_test_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

y_pred_train_xgb = xgb_model.predict(train_data_xgb)
y_pred_test_xgb = xgb_model.predict(test_data_xgb)

In [12]:
# 앙상블 예측 (평균이나 가중 평균 등의 방법 사용 가능)
y_pred_train_ensemble = (y_pred_train_lgb + y_pred_train_xgb) / 2
y_pred_test_ensemble = (y_pred_test_lgb + y_pred_test_xgb) / 2

In [13]:
# 앙상블 예측 평가
ensemble_rmse_train = mean_squared_error(y_train, y_pred_train_ensemble, squared=False)
ensemble_rmse_test = mean_squared_error(y_test, y_pred_test_ensemble, squared=False)
ensemble_r2_train = r2_score(y_train, y_pred_train_ensemble)
ensemble_r2_test = r2_score(y_test, y_pred_test_ensemble)

print(f'앙상블 훈련 RMSE: {ensemble_rmse_train}')
print(f'앙상블 테스트 RMSE: {ensemble_rmse_test}')
print(f'앙상블 훈련 R-squared: {ensemble_r2_train}')
print(f'앙상블 테스트 R-squared: {ensemble_r2_test}')

앙상블 훈련 RMSE: 1.7834876862737674
앙상블 테스트 RMSE: 1.9027432850722883
앙상블 훈련 R-squared: 0.619766323264174
앙상블 테스트 R-squared: 0.5669466042323753


In [14]:
lgb_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train_lgb))
lgb_r2_train = r2_score(y_train, y_pred_train_lgb)

lgb_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_lgb))
lgb_r2_test = r2_score(y_test, y_pred_test_lgb)

print("LightGBM 모델 평가 지표:")
print("훈련 RMSE:", lgb_rmse_train)
print("훈련 R-squared:", lgb_r2_train)
print("테스트 RMSE:", lgb_rmse_test)
print("테스트 R-squared:", lgb_r2_test)

LightGBM 모델 평가 지표:
훈련 RMSE: 1.9209784440725566
훈련 R-squared: 0.5588814392925394
테스트 RMSE: 1.9700359414401163
테스트 R-squared: 0.5357741163360898


In [15]:
xgb_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train_xgb))
xgb_r2_train = r2_score(y_train, y_pred_train_xgb)

xgb_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_xgb))
xgb_r2_test = r2_score(y_test, y_pred_test_xgb)

print("XGBoost 모델 평가 지표:")
print("훈련 RMSE:", xgb_rmse_train)
print("훈련 R-squared:", xgb_r2_train)
print("테스트 RMSE:", xgb_rmse_test)
print("테스트 R-squared:", xgb_r2_test)

XGBoost 모델 평가 지표:
훈련 RMSE: 1.7220080515194014
훈련 R-squared: 0.6455290051424385
테스트 RMSE: 1.9135301281431685
테스트 R-squared: 0.5620226399398447
